In [3]:
#!pip install beautifulsoup
#!wget https://dumps.wikimedia.org/enwiki/20190420/enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2
#!bunzip2 enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2    

In [4]:
# Count lines, words, characters in XML data file
!wc enwiki-20190420-pages-articles-multistream1.xml-p10p30302

 4586335 74563548 642819893 enwiki-20190420-pages-articles-multistream1.xml-p10p30302


In [5]:
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
def get_pages_from_xml_file(filename, start_tag='<page>', end_tag='</page>'):
    """Yields each page from the specified XML data file."""
    page = None
    with open(filename) as f:
        for line in f:
            if start_tag in line:
                page = []
                page.append(line)
            elif end_tag in line:
                page.append(line)
                page_xml = ''.join(page)
                yield page_xml
                page = None
            else:
                if page is not None:
                    page.append(line)

In [7]:
filename = 'enwiki-20190420-pages-articles-multistream1.xml-p10p30302'
pages = list(get_pages_from_xml_file(filename))

In [8]:
def get_title_from_page_xml(page_xml):
    """Returns the title of the given page."""
    soup = BeautifulSoup(page_xml, 'lxml') #, 'lxml'
    return soup.select_one('title').text

In [9]:
len(pages)  # Show the number of pages

19822

In [10]:
%time titles = [get_title_from_page_xml(page) for page in pages]

CPU times: user 48.7 s, sys: 4.53 s, total: 53.3 s
Wall time: 55.2 s


In [11]:
titles[:5] + ['...'] + titles[-5:]  # Show the first and last 5 titles

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople',
 '...',
 'The Lord of the Rings/One Ring',
 'Tax Freedom Day',
 'Tax',
 'Transhumanism',
 'TARDIS']

In [12]:
print(pages[0])  # Print the first page

  <page>
    <title>AccessibleComputing</title>
    <ns>0</ns>
    <id>10</id>
    <redirect title="Computer accessibility" />
    <revision>
      <id>854851586</id>
      <parentid>834079434</parentid>
      <timestamp>2018-08-14T06:47:24Z</timestamp>
      <contributor>
        <username>Godsy</username>
        <id>23257138</id>
      </contributor>
      <comment>remove from category for seeking instructions on rcats</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Computer accessibility]]

{{R from move}}
{{R from CamelCase}}
{{R unprintworthy}}</text>
      <sha1>42l0cvblwtb4nnupxm6wo000d27t6kf</sha1>
    </revision>
  </page>



In [15]:
def get_text_from_page_xml(page):
    """Returns the text of the given page."""
    soup = BeautifulSoup(page) #, 'lxml'
    return soup.select_one('text').text

In [21]:
page = pages[0]
soup = BeautifulSoup(page)
text = get_text_from_page_xml(page)
text, type(text)

('#REDIRECT [[Computer accessibility]]\n\n{{R from move}}\n{{R from CamelCase}}\n{{R unprintworthy}}',
 str)

In [22]:
data = []
for page in pages:
    row = {
        'title': get_title_from_page_xml(page),
        'text': get_text_from_page_xml(page),
    }
    data.append(row)

In [24]:
df = pd.DataFrame(data)

(19822, 2)

### Challenge

#### 1. Create a Pandas dataframe containing the title and text of each page.

* Implement the `get_text_from_page_xml` function above.
* Re-create the dataframe with the text field filled in.

#### 2. Identify the five pages that have the _longest_ text.

* Find the length of each page's `<text>...</text>` element and add it to your dataframe.
* Sort the data frame by text length, descending.
* What are the titles of the five longest articles?

In [25]:
titles = [get_title_from_page_xml(page) for page in pages]
text = [get_text_from_page_xml(page) for page in pages]

In [26]:
titles_series = pd.Series(titles)
text_series = pd.Series(text)


In [30]:
df['text'] = text_series

In [40]:
df.columns = ['title','text']

In [50]:
df['text_length'] = df.text.str.len()

In [63]:
df.sort_values('text_length',ascending=False).head()

,title,text,text_length
10165,List of compositions by Johann Sebastian Bach,{{short description|Wikipedia list page}}\n{{s...,432964
15129,Pakistan,{{about|the country}}\n{{pp-semi-indef}}\n{{pp...,376957
15287,Philippines,{{Redirect|Philippine|the town in the Netherla...,357950
9299,Foreign relations of India,{{Short description|Overview of relations}}{{U...,317725
8838,History of India,{{short description|Aspect of history}}\n{{pp-...,316816
